In [ ]:
#피보나치 되돌림
전략소개 : 각 매물대가 고점과 저점을 기준으로 해서 0.236, 0.318, 0.5, 0.618, 0.786에 지지와 저항을 받는 다는 것

In [ ]:
# 단순 지정 매수 매도
기간 설정 : 60일 동일
손절 : 저점 갱신시 손절
0.236 매수, 0.786 매도
초기 시드 1,000,000

90일
익절 횟수: 323
손절 횟수: 276
총 수익률: 1,294,212.15

80일
익절 횟수: 341
손절 횟수: 275
총 수익률: 1,989,134.90 = 198.9%의 수익
    
70일
익절 횟수: 344
손절 횟수: 280
총 수익률: 1,403,367.60

60일
익절 횟수: 349
손절 횟수: 296
총 수익률: -300607.43
    
50일
익절 횟수: 368
손절 횟수: 301
총 수익률: -70280.31
    
40일
익절 횟수: 377
손절 횟수: 305
총 수익률: 1,063,796.94 = 106.3%의 수익
    
30일
익절 횟수: 385
손절 횟수: 311
총 수익률: -32309.47

In [ ]:
import pandas as pd
import backtrader as bt
import time

# CSV 파일 경로
file_path = "./Recent_BTC_1day.csv"

# CSV 파일을 pandas 데이터프레임으로 불러오기
btc_data = pd.read_csv(file_path)

# 열 이름을 backtrader에서 사용하기 쉽게 변경
btc_data = btc_data.rename(columns={
    'Open_time': 'datetime',
    '시가': 'open',
    '고가': 'high', 
    '저가': 'low',
    '종가': 'close',
    '거래량': 'volume'
})

# backtrader에 필요 없는 열 제거
btc_data = btc_data[['datetime', 'open', 'high', 'low', 'close', 'volume']]

# datetime 컬럼을 datetime 타입으로 변환하고 인덱스로 설정
btc_data['datetime'] = pd.to_datetime(btc_data['datetime'])
btc_data.set_index('datetime', inplace=True)

# backtrader 데이터피드로 변환
class CustomData(bt.feeds.PandasData):
    params = (
        ('datetime', None),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1),
    )

data_feed = CustomData(dataname=btc_data)

# Cerebro 초기 설정 및 데이터 추가
cerebro = bt.Cerebro(optreturn=False, stdstats=False)
cerebro.broker.setcommission(commission=0.05 / 100, leverage=100)  # upbit 수수료인 0.05% 수수료 설정
cerebro.broker.setcash(100_0000) 
cerebro.broker.set_coc(True)  # 종가 진입 설정

# 데이터 피드 추가
cerebro.adddata(data_feed, name="BTC")

# 피보나치 되돌림 전략 구현
class FibonacciRetracement(bt.Strategy):
    params = dict(
        risk_ratio=100,  # 위험 비율 = 100%는 전체시드
        fib_236=0.236,  # 피보나치 되돌림 수준 (0.236)
        fib_786=0.786,  # 피보나치 되돌림 수준 (0.786)
        lookback_period=80, # 설정한 기간
    )

    def __init__(self):
        self.lowest_low = bt.indicators.Lowest(self.data.low, period=self.p.lookback_period)
        self.highest_high = bt.indicators.Highest(self.data.high, period=self.p.lookback_period)

        # 로그 파일 설정
        self.log_file = open('trading_log.txt', 'w')

    def next(self):
        # 60일 동안의 저점과 고점 계산
        lowest = self.lowest_low[0]
        highest = self.highest_high[0]

        # 피보나치 되돌림 수준 계산
        fib_236 = highest - (highest - lowest) * self.p.fib_236
        fib_786 = highest - (highest - lowest) * self.p.fib_786

        # 매수 조건 (0.236에서 매수)
        if self.data.close[0] <= fib_236 and not self.position:
            order_size = self.broker.getcash() * self.p.risk_ratio / 100 / self.data.close[0]
            self.buy(size=order_size)
            self.log(f"[매수 주문] 가격: {self.data.close[0]:.2f} 수량: {order_size:.10f}")

        # 매도 조건 (0.786에서 매도)
        elif self.data.close[0] >= fib_786 and self.position:
            self.sell(size=self.position.size)
            self.log(f"[매도 주문] 가격: {self.data.close[0]:.2f} 수량: {self.position.size:.10f}")

        # 손절 조건 (저점 갱신)
        elif self.position and self.data.low[0] < lowest:
            self.sell(size=self.position.size)
            self.log(f"[손절] 가격: {self.data.close[0]:.2f} 수량: {self.position.size:.10f}")

    def log(self, txt, dt=None):
        '''로깅 기능 (날짜 포함)'''
        dt = dt or self.datas[0].datetime.date(0)
        log_message = f'{dt.isoformat()} {txt}\n'
        print(log_message)  # 콘솔에도 출력
        self.log_file.write(log_message)  # 파일에 기록

    def stop(self):
        '''전략이 끝날 때 로그 파일 닫기'''
        self.log_file.close()


# 전략 추가
cerebro.addstrategy(FibonacciRetracement)

# pyfolio 분석기 추가
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

# 백테스트 실행
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

results = cerebro.run()

# 결과 출력
print(f'\nFinal Portfolio Value {cerebro.broker.getvalue()}')

# pyfolio 분석기 결과 가져오기
pyfoliozer = results[0].analyzers.getbyname('pyfolio')

# 결과 분석 및 시각화
import quantstats as qs
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
qs.plots.snapshot(returns)

# 결과 리포트 생성
print("\nGenerating report...")
import os
if not os.path.exists("./results"):
    os.makedirs("./results")
qs.reports.html(returns, output=f'./results/FibonacciRetracement_{int(time.time())}.html',
                download_filename=f'./results/FibonacciRetracement_{int(time.time())}.html',
                title="Fibonacci Retracement Strategy")
print("Complete")

In [ ]:
변동성에 따른 날자 변경 피보나치의 고점과 저점이 n% 이상 차이날 경우 40일로 변경
50%
익절 횟수: 401
손절 횟수: 326
총 수익률: 932833.37
75%
익절 횟수: 382
손절 횟수: 311
총 수익률: 175183.74
100%
익절 횟수: 384
손절 횟수: 294
총 수익률: 2776889.60
110%가 최고 수익률
익절 횟수: 372
손절 횟수: 287
총 수익률: 3552449.16

In [ ]:
import pandas as pd
import backtrader as bt
import time

# CSV 파일 경로
file_path = "./Recent_BTC_1day.csv"

# CSV 파일을 pandas 데이터프레임으로 불러오기
btc_data = pd.read_csv(file_path)

# 열 이름을 backtrader에서 사용하기 쉽게 변경
btc_data = btc_data.rename(columns={
    'Open_time': 'datetime',
    '시가': 'open',
    '고가': 'high', 
    '저가': 'low',
    '종가': 'close',
    '거래량': 'volume'
})

# backtrader에 필요 없는 열 제거
btc_data = btc_data[['datetime', 'open', 'high', 'low', 'close', 'volume']]

# datetime 컬럼을 datetime 타입으로 변환하고 인덱스로 설정
btc_data['datetime'] = pd.to_datetime(btc_data['datetime'])
btc_data.set_index('datetime', inplace=True)

# backtrader 데이터피드로 변환
class CustomData(bt.feeds.PandasData):
    params = (
        ('datetime', None),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume'),
        ('openinterest', -1),
    )

data_feed = CustomData(dataname=btc_data)

# Cerebro 초기 설정 및 데이터 추가
cerebro = bt.Cerebro(optreturn=False, stdstats=False)
cerebro.broker.setcommission(commission=0.05 / 100, leverage=100)
cerebro.broker.setcash(100_0000)
cerebro.broker.set_coc(True)

# 데이터 피드 추가
cerebro.adddata(data_feed, name="BTC")

# 피보나치 되돌림 전략 구현
class FibonacciRetracement(bt.Strategy):
    params = dict(
        risk_ratio=100,
        fib_236=0.236,
        fib_786=0.786,
        default_lookback=80,
        reduced_lookback=40,
        threshold=1.1  # 110% 차이 기준
    )

    def __init__(self):
        self.lookback_period = self.p.default_lookback
        self.lowest_low = bt.indicators.Lowest(self.data.low, period=self.lookback_period)
        self.highest_high = bt.indicators.Highest(self.data.high, period=self.lookback_period)

    def next(self):
        lowest = self.lowest_low[0]
        highest = self.highest_high[0]
        price_difference_ratio = (highest - lowest) / lowest

        # 50% 이상 차이 나면 lookback_period 변경
        if price_difference_ratio > self.p.threshold:
            self.lookback_period = self.p.reduced_lookback
        else:
            self.lookback_period = self.p.default_lookback

        # 피보나치 되돌림 수준 계산
        fib_236 = highest - (highest - lowest) * self.p.fib_236
        fib_786 = highest - (highest - lowest) * self.p.fib_786

        # 매수 조건
        if self.data.close[0] <= fib_236 and not self.position:
            order_size = self.broker.getcash() * self.p.risk_ratio / 100 / self.data.close[0]
            self.buy(size=order_size)
            self.log(f"[매수 주문] 가격: {self.data.close[0]:.2f} 수량: {order_size:.10f}")

        # 매도 조건
        elif self.data.close[0] >= fib_786 and self.position:
            self.sell(size=self.position.size)
            self.log(f"[매도 주문] 가격: {self.data.close[0]:.2f} 수량: {self.position.size:.10f}")

        # 손절 조건
        elif self.position and self.data.low[0] < lowest:
            self.sell(size=self.position.size)
            self.log(f"[손절] 가격: {self.data.close[0]:.2f} 수량: {self.position.size:.10f}")

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}')

# 전략 추가
cerebro.addstrategy(FibonacciRetracement)

# 백테스트 실행
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print(f'\nFinal Portfolio Value {cerebro.broker.getvalue()}')


In [ ]:
되돌림 비율이 언제 효과 적일지 테스트 해본 결과
0.118, 0.786
익절 횟수: 444
손절 횟수: 337
총 수익률: 11,625,837.44 = 1162.58%
수수료 제외 : 9,038,844.30 = 903.88%
익절 확률: 56.85019206145967%

0.118, 0.789
익절 횟수: 477
손절 횟수: 361
총 수익률: 15,292,150.21 = 1529.21%
수수료 제외 : 11,764,803.67 = 1176.48% 
익절 확률: 56.921241050119335%

In [ ]:
결론 : 매도는 0.783~0.792 로 피보나치 비율을 사용하는 0.786 부근이 효과 적이였고 다음으로 활용하는 0.83 부근도 꽤나 높은 수익률을 보였다
매수는 0.118 부근에서 가장 효과적이였는데 이유는 저점을 갱신하고 안정을 찾기 시작하는 부분이 0.118인 것 같다 그리고 손익비가 1:7 정도이므로 익절확률이 절반만 되어도 성공을 하는 것 같다
그 다음으로 0.118,0.119,0.117 다음으로 0.03도 의미가 있었는데 최저점으로 3% 증가해서 높은 수익을 보였는데 손절확률이 높아 수수료로 많이 깍인 듯 하다.
손익 비율은 1 : 26이므로 나중에 단타를 생각하면 나쁘지 않은 손익비로 생각된다